# 1. Preliminaries

In [1]:
!pip install qutip

In [2]:
import numpy as np
import qutip as qp
import scipy as sp

# 2. Gell_Mann matrix generator function

this function will generate the gellmann matrix of dimension d.

the function "gellmann(j,k,d)" will generate the jth_kth gellmann matrix of dimension d. so for every d dimension we have d^2 gellmann matrix. one of them is I, so the number of gellmann matrix that is not trivial is (d^2 - 1).

In [3]:
"""Generate generalized Gell-Mann matrices.
  .. module:: gellmann.py
     :synopsis: Generate generalized Gell-Mann matrices
  .. moduleauthor:: Jonathan Gross <jarthurgross@gmail.com>
"""
from itertools import product

def gellmann(j, k, d):
    r"""Returns a generalized Gell-Mann matrix of dimension d.
    
    According to the convention in *Bloch Vectors for Qubits* by Bertlmann and
    Krammer (2008), returns :math:`\Lambda^j` for :math:`1\leq j=k\leq d-1`,
    :math:`\Lambda^{kj}_s` for :math:`1\leq k<j\leq d`, :math:`\Lambda^{jk}_a`
    for :math:`1\leq j<k\leq d`, and :math:`I` for :math:`j=k=d`.
    Parameters
    ----------
    j : positive integer
        Index for generalized Gell-Mann matrix
    k : positive integer
        Index for generalized Gell-Mann matrix
    d : positive integer
        Dimension of the generalized Gell-Mann matrix
    Returns
    -------
    numpy.array
        A genereralized Gell-Mann matrix.
    """

    if j > k:
        gjkd = np.zeros((d, d), dtype=np.complex128)
        gjkd[j - 1][k - 1] = 1
        gjkd[k - 1][j - 1] = 1
    elif k > j:
        gjkd = np.zeros((d, d), dtype=np.complex128)
        gjkd[j - 1][k - 1] = -1.j
        gjkd[k - 1][j - 1] = 1.j
    elif j == k and j < d:
        gjkd = np.sqrt(2/(j*(j + 1)))*np.diag([1 + 0.j if n <= j
                                               else (-j + 0.j if n == (j + 1)
                                                     else 0 + 0.j)
                                               for n in range(1, d + 1)])
    else:
        gjkd = np.diag([1 + 0.j for n in range(1, d + 1)])

    return gjkd

def get_basis(d):
    r"""Return a basis of operators.
    
    The basis is made up of orthogonal Hermitian operators on a Hilbert space
    of dimension d, with the identity element in the last place.
    Parameters
    ----------
    d : int
        The dimension of the Hilbert space.
    Returns
    -------
    list of numpy.array
        The basis of operators.
    """
    return [gellmann(j, k, d) for j, k in product(range(1, d + 1), repeat=2)]

in this cell the gellmann matrix of 3 dimension will generate and they will save in a dictionary and labeled them by numbers 0,1,...,8 .

In [4]:

def gellmann_basis(d):
    gellmann_basis = {}  ## a dictionary that saved the gellmann matrices

    k=0
    for i in range(1,d+1):
        for j in range(1,d+1):
            gellmann_basis.update({"G{}".format(k):gellmann(i,j,d)})
            k+=1
        
    return gellmann_basis

gellmann_basis(3)

{'G0': array([[ 1.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j]]),
 'G1': array([[ 0.+0.j, -0.-1.j,  0.+0.j],
        [ 0.+1.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j]]),
 'G2': array([[ 0.+0.j,  0.+0.j, -0.-1.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+1.j,  0.+0.j,  0.+0.j]]),
 'G3': array([[0.+0.j, 1.+0.j, 0.+0.j],
        [1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j]]),
 'G4': array([[ 0.57735027+0.j,  0.        +0.j,  0.        +0.j],
        [ 0.        +0.j,  0.57735027+0.j,  0.        +0.j],
        [ 0.        +0.j,  0.        +0.j, -1.15470054+0.j]]),
 'G5': array([[ 0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j, -0.-1.j],
        [ 0.+0.j,  0.+1.j,  0.+0.j]]),
 'G6': array([[0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j],
        [1.+0.j, 0.+0.j, 0.+0.j]]),
 'G7': array([[0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j]]),
 'G

# 3. Coefficients of gellmann's expansion

at this cell a random density matrix will generate by qutip library.

for calculating the coefficients of gellmann expansion we do this iteration:
1. calculate the tensor product of gellmann matrices for every coefficient
2. calculate the dot product of "ro" and the "gellmann's tensor product"
3. calculate the trace of this matrix and divide it into 4
4. append it to our index list

In [12]:

ro = qp.rand_dm(9) ## generate the random density matrix
def coef_of_gellmann(ro):

    coef = []  ## the coefficient's dictionary (suitable format for saving as a data)
    gellmann3_basis = gellmann_basis(3) ## the gellmann matrix for a 3 dimension system
    
    for i in range(9):
        for j in range(9):
            tensorP = np.kron(gellmann3_basis["G{}".format(i)],gellmann3_basis["G{}".format(j)]) ## Tensor product of gelllmann matrices
            c = np.trace(np.dot(ro,tensorP))/4   ## dot product and trace
            c = c.real       ## all of gellmann coefs is a real
            c = round(c,8)  
            coef.append(c)

    index = np.array(coef)  ## change list to np.array() object
    return coef
print(coef_of_gellmann(ro))
ro

[-0.02105555, 0.00203751, 0.01979332, 0.02250977, -0.01482898, -0.00981834, -0.00197323, 0.01643552, 0.02248114, 0.00273691, -0.0339497, -0.00431171, -0.02281059, -0.02508618, -0.02480655, 0.02565827, -0.00043728, -0.03804276, -0.00676277, 0.01952881, -0.01336973, 0.00450483, 0.01074206, -0.01053976, -0.00910448, -0.0145112, 0.0211929, -0.00265626, -0.02603226, -0.01166996, -0.01630973, -0.00929702, -0.00080423, 0.00585442, -0.01093312, -0.00777188, -0.00385933, -0.00731328, 0.00717315, 0.01082607, 0.0065237, 0.01928492, 0.01953336, 0.00873372, 0.01112604, -0.01678663, -0.03560536, -0.00568023, -0.03404053, -0.01093016, -0.02621029, 0.03229175, 0.01624051, -0.0128741, 0.00137986, -0.00545632, -0.01956956, -0.01988996, -0.01132693, -0.03890426, 0.00714573, 0.00455901, -0.03863911, -0.01062942, 0.00223974, -0.00060518, 0.0023097, -0.00739486, 0.00399945, 0.00178301, 0.01282992, -0.01428926, -0.00485833, 0.01748752, 0.02124445, 0.02114959, 0.00020992, 0.00900506, -0.00608047, -0.01261106,

Quantum object: dims = [[9], [9]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[ 0.09755218+0.j          0.04285993-0.00947353j  0.00525071-0.03809771j
  -0.01320515+0.03710845j  0.01763997+0.04884285j  0.01016612-0.01398831j
  -0.03091916-0.01356777j -0.03941877+0.00095149j  0.02051546+0.02867404j]
 [ 0.04285993+0.00947353j  0.15059743+0.j          0.01307056-0.00731918j
  -0.05025943-0.00322168j -0.00789262+0.04258226j  0.01387343+0.00124151j
  -0.00036115-0.00996116j -0.03367888-0.0270933j   0.01509877+0.05341546j]
 [ 0.00525071+0.03809771j  0.01307056+0.00731918j  0.14299326+0.j
   0.00154271-0.03732823j -0.03573968-0.00036694j  0.00555402-0.00360518j
  -0.006224  -0.01046508j -0.00598075-0.02439306j -0.01268019-0.00172473j]
 [-0.01320515-0.03710845j -0.05025943+0.00322168j  0.00154271+0.03732823j
   0.12681145+0.j         -0.00215961-0.00539851j  0.00919718+0.00148894j
  -0.02442502+0.0316799j   0.03791506+0.0318008j   0.00746324-0.03168657j]
 [ 0.01763997-0.04884285j 

this cell is just a test of this algotithm in 2 dimension systems.

In [13]:
pauli1= np.array([[0,1],[1,0]])
##pauli2= np.array([0,0-1j],[0+1j,0])
pauli3= np.array([[1,0],[0,-1]])

a= qp.rand_dm(2)
c = np.trace(np.dot(a,pauli1))/2
print(c)
a

81
[ 0.03411602 -0.01282055 -0.04959134 -0.0162137   0.05330422 -0.02567253
  0.01301234 -0.011233   -0.01193508  0.03804849  0.03271166  0.03399261
  0.00064593  0.02196731  0.         -0.07369177  0.          0.03804849
  0.02744585  0.03564444 -0.1089237  -0.08561629  0.01584587  0.
  0.04303734  0.          0.02744585 -0.04621325  0.00064593 -0.07369177
 -0.03271166 -0.02668123  0.         -0.03399261  0.         -0.04621325
  0.05791614 -0.071229    0.0985962   0.01022413  0.05002959 -0.08013022
  0.06296194  0.09870324 -0.10388679 -0.01751242  0.03925517  0.03743489
  0.04703118 -0.05569015  0.1166844  -0.12430102 -0.01260716  0.11136204
 -0.06393658 -0.08561629  0.04303734 -0.03564444 -0.0369138   0.
  0.1089237   0.         -0.06393658  0.0304826   0.06510988 -0.00515788
  0.00526502  0.11490565 -0.01859369 -0.03425105  0.04206313 -0.01862137
  0.0407979   0.08091695 -0.01099981  0.01546619 -0.04122513  0.10790361
 -0.07404516 -0.06863001  0.25      ]


Making a Convex Hull for a 2_qutrit system and checking for new points in the convex Hull

In [15]:
d_sub1 = 3
d_sub2 = 3
num_extreme_points = 83
index = np.zeros((1,81))

for i in range(num_extreme_points):
    ro = qp.rand_dm(d_sub1 * d_sub2, pure=True)

    coef = []
    coef = coef_of_gellmann(ro)
    coef = np.reshape(np.array(coef),(1,81))
    index = np.vstack((index,coef))
#print(np.shape(index[1:,]))

Using different functions to build convex hull

In [8]:
#generating convex hull from extreme points(Pure separable states)
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
#hull = Delaunay(index)
#hull = ConvexHull(index)
#print(index)

The function that checks if a point is in a convex hull

In [9]:
#This function checks if a point is in the convex hull or not
def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    from scipy.spatial import Delaunay
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

Test

In [16]:
ro = qp.rand_dm(9) ## generate the random density matrix

coef = []
coef = coef_of_gellmann(ro)
coef = np.reshape(np.array(coef),(1,81)) 

print(coef)
#print(in_hull(index[20:,],index))

[[-2.424698e-02  3.898820e-03 -2.062850e-02  2.063487e-02  8.481900e-03
   4.084100e-03 -3.510745e-02  2.384312e-02 -1.035160e-03  7.761220e-03
   7.764620e-03 -1.093209e-02  3.876805e-02 -4.273177e-02  1.548906e-02
   1.507003e-02  3.720262e-02 -2.360800e-04  1.768104e-02  6.473400e-04
   1.514866e-02 -1.602554e-02 -2.452260e-02  5.229260e-03 -1.250279e-02
   8.804920e-03  1.360203e-02  1.299680e-03 -4.001684e-02  2.317710e-03
   4.694300e-04  2.622235e-02  3.185035e-02  1.222665e-02 -2.532996e-02
  -1.434948e-02  8.139240e-03 -2.175503e-02 -7.824510e-03 -1.292924e-02
  -3.345510e-03 -5.968860e-03  3.191260e-02  4.710328e-02  6.467630e-03
   1.114629e-02  1.388967e-02  9.291840e-03 -1.888272e-02  1.541750e-03
  -2.850930e-03 -1.575407e-02  3.361019e-02 -1.571077e-02  2.242580e-03
   2.500200e-04  9.250840e-03 -2.643100e-04 -5.306510e-03 -9.971970e-03
  -1.261009e-02  1.300286e-02  6.214280e-03 -1.374033e-02  1.438176e-02
  -4.085130e-03 -2.130565e-02 -6.761900e-04  1.983930e-02  1.563

Making a Convex Hull for a 2_qubit system and checking for new points in the convex Hull

In [10]:
gellmann2_basis = {}

k=0
for i in range(1,3):
    for j in range(1,3):
        gellmann2_basis.update({"G{}".format(k):gellmann(i,j,2)})
        k+=1
        
gellmann2_basis

{'G0': array([[ 1.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j]]), 'G1': array([[ 0.+0.j, -0.-1.j],
        [ 0.+1.j,  0.+0.j]]), 'G2': array([[0.+0.j, 1.+0.j],
        [1.+0.j, 0.+0.j]]), 'G3': array([[1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]])}

In [15]:
d_sub1 = 2
d_sub2 = 2
num_extreme_points = 20
index = np.zeros((1,16))

for i in range(num_extreme_points):
    #psi_sub1 = qp.rand_ket(d_sub1)
    #ro_sub1  = psi_sub1*psi_sub1.dag()
    #psi_sub2 = qp.rand_ket(d_sub2)
    #ro_sub2  = psi_sub2*psi_sub2.dag()
    ro = qp.ran_dm(d_sub1 * d_sub2, pure=True)

    coef = []
    for i in range(d_sub1 * d_sub2):
        for j in range(d_sub1 * d_sub2):
            tensorP = np.kron(gellmann2_basis["G{}".format(i)],gellmann2_basis["G{}".format(j)])
            c = np.trace(np.dot(ro,tensorP))/4
            c = c.real
            c = round(c,6)
            coef.append(c)
    coef = np.reshape(np.array(coef),(1,16))
    index = np.vstack((index,coef))
#print(np.real(index[1:,]))

AttributeError: module 'qutip' has no attribute 'ran_dm'

In [15]:
#generating convex hull from extreme points(Pure separable states)
from scipy.spatial import ConvexHull
hull = ConvexHull(index[:,])
#print(index)

In [16]:
def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    from scipy.spatial import Delaunay
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

In [17]:
in_hull(index[2,],index)

True